# Writing a shell script to submit a job to TSCC cluster

## Make a template shell script

First we will make fake template submitter script to reference for all of our other submission jobs. Since you will likely use a variation of the same PBS commands for all your processing needs, you can make a script that you copy and modify for each job as necessary. We are going to use Bash to submit these scripts to the cluster, so they have the file extension `.sh` 

```
$ cd ~/
$ mkdir scripts
$ cd scripts
$ vi template.sh
i
#!/bin/bash
#PBS -q hotel
#PBS -N jobname
#PBS -l nodes=1:ppn=8
#PBS -l walltime=1:00:00
#PBS -o outputfile
#PBS -e errorfile
esc
:wq
```

You can learn more about which submission parameters to include and what they do [here](http://www.sdsc.edu/support/user_guides/tscc-quick-start.html).

## Make a shell script to submit STAR alignment as a job

Since running STAR generate genome index with the human reference genome requires a lot of processing power, we would need to submit this command as a job to the cluster. Remember that handy fake submission script we made? Let's use it here by copying it and updating the necessary parameters:
    
Copy our template script into a new file with a meaningful name

```
$ cp ~/scripts/template.sh ~/scripts/star_index_genome.sh
```
    
#### Q: 
What do you need to change in the PBS flags for this script?

#### Q:
How do you send yourself an email once the job is completed?

*HINT - Remember you can read about submission parameters [here](http://www.sdsc.edu/support/user_guides/tscc-quick-start.html)*

For this script, we will use a walltime of 3 hours, 1 node, and 16 processors.

Take your STAR generate genome index command, add it to your .sh file below all the PBS flags (that you have already modified above to make unique for this script). 


My finished `~/scripts/star_index_genome.sh` script looks like this:

```
#!/bin/bash
#PBS -q hotel 
#PBS -N jobname
#PBS -l nodes=1:ppn=16
#PBS -l walltime=1:00:00
#PBS -o outputfile.out
#PBS -e errorfile.err
#PBS -M mragsac@eng.ucsd.edu 

STAR --runThreadN 16 --runMode genomeGenerate --genomeDir ~/cmm262/references/yeast/star --genomeFastaFiles ~/cmm262/references/yeast/yeast.fa --sjdbGTFfile ~/cmm262/references/yeast/yeast.gtf
```




## Submit your script to the cluster with:

```
$ qsub star_index_genome.sh
```

## How to check the status of your job

```
$ qstat -u username
```
    
AKA:

```
$ qstat -u ucsd-train01
```
    
Take a look at the status (The column labeled S). Q means your job is in the queue and has not started yet. R means your job is running (you will see the time updated according to how long it has been running). C means your job is complete. 

Once your job has been running for ~5-10 minutes without aborting, you likely are okay and it will run to completion. But this takes some time. So in the meantime, read up on the STAR mapping steps described below and write your script for mapping. However, you will have to wait until the generate genome step is complete before you submit your mapping job.

## How to delete your job

If you realize after you submitted your script that you made a mistake and would like to delete your job, you can do that with:

```
qdel jobid##
```
    
You can get the jobID# from the output of:

```
qstat -u username
```

**Making Aliases**

If there are particular commands that you use a lot but are lengthy to type each time, you can make a handy shortcut for yourself by defining an alias in your ~/.bashrc file. For example, 

    $qstat -u ucsd-train01
    
is really annoying to type and I use it a lot. I want to make a shortcut so I can just use:

    $ qme
    
To do this, open your bashrc and add the following line to the bottom of your file, BELOW the line that says #user specific aliases and functions:

    $ vi ~/.bashrc
    i
    alias qme="qstat -u ucsd-train##"
    esc
    :wq
    
*NOTE - Don't forget to substitute your specific number for the ##*

Now try your new alias!

    $ qme
    
What happened? Why did you get this error?

    -bash: qme: command not found


Your ~/.bashrc will only be read at the beginning of a login session. Here, we changed our ~/.bashrc, but have not logged out and logged back in. In order to activate the changes that you made to the file you will need to source your ~/.bashrc:

    $ source ~/.bashrc
    
Now try it again:

    $ qme